                                            ETL para user_reviews.json

In [1]:
import pandas as pd
import json
%load_ext autoreload
%autoreload 2
import numpy as np
import re
import ast

import warnings
warnings.filterwarnings("ignore")

En primer lugar se lee el archivo proporcionado para concer un poco más sobre su contenido:

In [2]:
#Cargar el archivo json
filas = []
# open file json
with open('../Json iniciales/australian_user_reviews.json', 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        filas.append(ast.literal_eval(line))
# create data frame with data
df_user_reviews = pd.DataFrame(filas)
# display df
df_user_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [3]:
df_user_reviews2 = df_user_reviews

In [4]:
#De una vez eliminamos la columna user_url, puesto que no será útil en las consultas de la API
df_user_reviews2 = df_user_reviews2.drop(columns=['user_url'])

In [5]:
def tipo_de_Dato_df(dataframe):
    datos = {"Columna": [], "tipo_de_dato": [], "%_no_nulos": [], "%_nulos": [], "nulos": []}

    for columna in dataframe.columns:
        no_nulos = (dataframe[columna].count() / len(dataframe)) * 100
        datos["Columna"].append(columna)
        datos["tipo_de_dato"].append(dataframe[columna].apply(type).unique())
        datos["%_no_nulos"].append(round(no_nulos, 2))
        datos["%_nulos"].append(round(100-no_nulos, 2))
        datos["nulos"].append(dataframe[columna].isnull().sum())

    df_info = pd.DataFrame(datos)
        
    return df_info

tipo_de_Dato_df(df_user_reviews2)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,user_id,[<class 'str'>],100.0,0.0,0
1,reviews,[<class 'list'>],100.0,0.0,0


Se verifican si existen duplicados:

In [6]:
def duplicados(dataframe, columna):
    #Buscamos los duplicados en la columna y los guardamos, si no existen, retornamos 'sin duplicados'
    duplicados = dataframe[dataframe.duplicated(subset=columna, keep=False)]
    if duplicados.empty:
        return "Sin duplicados"

    duplicados_ordenados = duplicados.sort_values(by=columna)
    return duplicados_ordenados

duplicados(df_user_reviews2,'user_id')

,user_id,reviews
12888,05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
5250,05041129,"[{'funny': '', 'posted': 'Posted May 18, 2015...."
3133,111222333444555666888,"[{'funny': '', 'posted': 'Posted December 22, ..."
3134,111222333444555666888,"[{'funny': '', 'posted': 'Posted December 22, ..."
4139,29123,"[{'funny': '', 'posted': 'Posted March 26.', '..."
...,...,...
2721,xXAussieRockXx,"[{'funny': '', 'posted': 'Posted July 17, 2015..."
2680,yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
17916,yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
5855,zeroblade,"[{'funny': '', 'posted': 'Posted November 30, ..."


Sin embargo, sabemos que cada usuario puede dar diferentes reviews, por lo tanto, debemos saber si lo que se repite es la review en ese mismo juego, por el mismo usuario:

In [7]:


reviews_duplicadas = duplicados(df_user_reviews2, 'user_id')

# Se selecciona las reviews de un usuariode ejemplo ('yolofaceguy')
user_id = 'yolofaceguy'
user_reviews = reviews_duplicadas[reviews_duplicadas['user_id'] == user_id]['reviews']

# Se itera sobre las listas de reviews del usuario de ejemplo
for review_list in user_reviews:
    for review in review_list:
        print(review['review'])
    print('\n')


from the creaters of the walking dead, i present to you, the wolf among us. a twisted an unhappy place where crimes are made in the town of the fables. Ecperience one of the most mind-bending, jaw-dropping twists you will see in all of gaming history...well, some of gaming history. SPOILERS: i really liked how bigby turns into his true form to fight all the bloody marys... its just like when neo fought all the agent smiths in the matrix. what i didnt really like is when i chose to lock the crooked man up it glitches my game and i start from the near begining where bigby fought the crooked man's crew, exept the guy who runs a strip club wasnt there. so i was really confused. but anyway i really recommend this game if you want twists and ♥♥♥♥ed up scenes.
this game is awesome,this game is ♥♥♥♥ed up and this game is so sad and depressing. if you want these types of games i would strongly reccomend youto buy it. its worth your money


from the creaters of the walking dead, i present to you

In [8]:
#Dropeamos los duplicados, dejando la primera aparicion de cada review.
df_user_reviews2 = df_user_reviews2.drop_duplicates(subset='user_id', keep='first')


In [9]:
def duplicados(dataframe, columna):
    duplicados = dataframe[dataframe.duplicated(subset=columna, keep=False)]
    if duplicados.empty:
        return "Sin duplicados"

    duplicados_ordenados = duplicados.sort_values(by=columna)
    return duplicados_ordenados

duplicados(df_user_reviews2,'user_id')

'Sin duplicados'

La columna reviews necesita ser desanidada, ya que contiene diccionarios, cuyas llaves son:

    funny: emoticon gracioso
    posted: fecha cuando se realizó la review
    last_edited: última edición
    item_id: id del juego
    helpful: indica si la review fue útil o no
    recommend: determina si el usuario recomienda o no el juego
    review: cadena de texto con la reseñadel juego

In [10]:
df_user_reviews2 = df_user_reviews2.explode('reviews') 
df_user_reviews2 = pd.concat([df_user_reviews2.drop(['reviews'], axis=1), df_user_reviews2['reviews'].apply(pd.Series)], axis=1)

In [11]:
df_user_reviews3=df_user_reviews2


In [12]:

df_user_reviews3.head()

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN


In [13]:
tipo_de_Dato_df(df_user_reviews3)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,funny,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
2,posted,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
3,last_edited,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
4,item_id,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
5,helpful,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
6,recommend,"[<class 'bool'>, <class 'float'>]",99.95,0.05,28
7,review,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
8,0,[<class 'float'>],0.00,100.00,58458


In [14]:
#Al hacer el explode se crea una columna llamda 0, esta columna no es útil y solo contiene valores nulos:
df_user_reviews3 = df_user_reviews3.rename(columns={0: 'a'})
df_user_reviews3 = df_user_reviews3.drop(columns=['a'])
df_user_reviews3

,user_id,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
0,76561197970982479,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
0,76561197970982479,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
1,js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
1,js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...
25797,76561198312638244,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...
25797,76561198312638244,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...
25798,LydiaMorley,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
25798,LydiaMorley,,Posted July 20.,,730,No ratings yet,True,:D


In [15]:
tipo_de_Dato_df(df_user_reviews3)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,user_id,[<class 'str'>],100.00,0.00,0
1,funny,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
2,posted,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
3,last_edited,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
4,item_id,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
5,helpful,"[<class 'str'>, <class 'float'>]",99.95,0.05,28
6,recommend,"[<class 'bool'>, <class 'float'>]",99.95,0.05,28
7,review,"[<class 'str'>, <class 'float'>]",99.95,0.05,28


Se dropean las columnas qué no serán útiles para la api:

In [16]:
#columnas a dropear:
df_user_reviews3 = df_user_reviews3.drop(['funny', 'last_edited', 'helpful'], axis=1)
df_user_reviews3.columns


Index(['user_id', 'posted', 'item_id', 'recommend', 'review'], dtype='object')

Renombramos las columnas que si usaremos:


In [17]:
nuevos_nombres = {'user_id': 'User_Id', 'posted': 'Posted','item_id':'Item_Id',
                  'recommend':'Recommend', 'review': 'Review'}
df_user_reviews3 = df_user_reviews3.rename(columns=nuevos_nombres)

df_user_reviews3.columns

Index(['User_Id', 'Posted', 'Item_Id', 'Recommend', 'Review'], dtype='object')

In [18]:
#Se limpian los nulos en las columnas
df_user_reviews3.dropna(subset = ['Posted'], inplace=True)
df_user_reviews3.dropna(subset = ['Item_Id'], inplace=True)
df_user_reviews3.dropna(subset = ['Recommend'], inplace=True)
df_user_reviews3.dropna(subset = ['Review'], inplace=True)                  


In [19]:
tipo_de_Dato_df(df_user_reviews3)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,User_Id,[<class 'str'>],100.0,0.0,0
1,Posted,[<class 'str'>],100.0,0.0,0
2,Item_Id,[<class 'str'>],100.0,0.0,0
3,Recommend,[<class 'bool'>],100.0,0.0,0
4,Review,[<class 'str'>],100.0,0.0,0


                                            COLUMNA POSTED

Como podemos observar, la columna Posted, cuenta con esta palabra, previa al mes, dia y año de posteo de la review, se trabajará esta columna para obtener solamente el año de posteo de la reseña, puesto que es lo que necesitaremos posteriormente para la api.

In [20]:
df_user_reviews3['Posted'][0]

0    Posted November 5, 2011.
0       Posted July 15, 2011.
0      Posted April 21, 2011.
Name: Posted, dtype: object

In [21]:
#Se reemplaza la palabra 'Posted' por un espacio:
df_user_reviews3['Posted'] = df_user_reviews3['Posted'].str.replace('Posted ', '')
#Pasamos a formato date time:
df_user_reviews3['Posted'] = pd.to_datetime(df_user_reviews3['Posted'],dayfirst=True, errors='coerce')

In [22]:
tipo_de_Dato_df(df_user_reviews3)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,User_Id,[<class 'str'>],100.0,0.0,0
1,Posted,[<class 'pandas._libs.tslibs.timestamps.Timest...,83.0,17.0,9932
2,Item_Id,[<class 'str'>],100.0,0.0,0
3,Recommend,[<class 'bool'>],100.0,0.0,0
4,Review,[<class 'str'>],100.0,0.0,0


In [23]:
#Eliminamos los datos not a time:
df_user_reviews3.dropna(subset = ['Posted'], inplace=True)

In [24]:
#Pasamos la columna a formato año - mes - día:
df_user_reviews3['Posted'] = df_user_reviews3['Posted'].dt.strftime('%Y-%m-%d')

In [25]:
#Se crea la columna Year_Posted:
df_user_reviews3['Year_Posted'] = pd.to_datetime(df_user_reviews3['Posted']).dt.year

df_user_reviews3.columns

Index(['User_Id', 'Posted', 'Item_Id', 'Recommend', 'Review', 'Year_Posted'], dtype='object')

In [26]:
#Se elimina la columna 'Posted'
df_user_reviews3 = df_user_reviews3.drop('Posted', axis=1)

In [27]:
#Verificamos el df:
tipo_de_Dato_df(df_user_reviews3)

,Columna,tipo_de_dato,%_no_nulos,%_nulos,nulos
0,User_Id,[<class 'str'>],100.0,0.0,0
1,Item_Id,[<class 'str'>],100.0,0.0,0
2,Recommend,[<class 'bool'>],100.0,0.0,0
3,Review,[<class 'str'>],100.0,0.0,0
4,Year_Posted,[<class 'int'>],100.0,0.0,0


In [28]:
df_user_reviews3['Item_Id'] = df_user_reviews3['Item_Id'].astype('int32')

In [29]:
#SE GUARDA EL DATAFRAME RESULTANTE:
user_reviews_limpio = '../CSV/ETL_user_review.csv'

df_user_reviews3.to_csv(user_reviews_limpio, index=False, encoding='utf-8')